## Accessing DM-EFD data


In this notebook we demonstrate how to extract data from the DM-EFD using [aioinflux](https://aioinflux.readthedocs.io/en/stable/index.html), a Python client for InfluxDB, and proceed with data analysis using Pandas dataframes. 

This is complementaty to the [Chronograf](https://test-chronograf-efd.lsst.codes) interface which we use for time-series visualization.

In addition to `aioinflux`, you'll need to install `pandas`, `numpy` and `matplotlib` to run this notebook.

In [ ]:
import matplotlib
%matplotlib widget
from matplotlib import pylab as plt
import aioinflux
import getpass
import pandas as pd
import asyncio

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d
output_notebook()

We'll access the DM-EFD instance deployed at the AuxTel lab in Tucson. You need to be on site or connected to the NOAO VPN. 

If you are familiar with the AuxTel lab environment, you might be able to authenticate using the generic `saluser`. Ping me at Slack (`@afausti`) if you have any problem.

In [ ]:
username = "saluser"
password = getpass.getpass(f"Password for {username}:")

The following configures the `aioinflux` Python client to connect to the DM-EFD InfluxDB instance. 

In [ ]:
client = aioinflux.InfluxDBClient(host='summit-influxdb-efd.lsst.codes', 
                                  port='443', 
                                  ssl=True, 
                                  username=username, 
                                  password=password,
                                  db='efd')

We can configure the output to be a Pandas dataframe, which is very convenient for data analysis.  Specify a time range for data in `InfluxQL`.  The default is 20hrs ago, but this may need to be changed depending on how recently data was taken.

In [ ]:
client.output = 'dataframe'
#time_span = "time >= '2019-09-07T21:30:00Z' AND time < '2019-09-08T06:00:00Z'"
time_span = "time >= '2019-09-08T01:41:00Z' AND time <= '2019-09-08T01:44:00Z'"

Query each of the measurements we may want to correlate later in the notebook.  Note that this could be done as a single query, but the result is a dictionary of `DataFrames` which I find less convenient to use than named variables corresponding to one `DataFrame` each.

In [ ]:
async def get_data_frame(field_base, topic, els=100):
    fields = ", ".join([f'"{field_base}{i}"' for i in range(100)])
    df = await client.query(f'SELECT "cRIO_timestamp", "private_sndStamp", {fields} FROM "efd"."autogen"."{topic}" WHERE {time_span}')

    times = []
    timestamps = []
    vals = []
    step = 1./els
    for row in df.itertuples():
        for i in range(els):
            times.append(row.cRIO_timestamp + i*step)
            timestamps.append((pd.Timestamp(row.cRIO_timestamp, unit='s', tz='GMT') + pd.Timedelta(i*step, unit='s')))
            vals.append(getattr(row, f'{field_base}{i}'))

    return pd.DataFrame({'times':times, field_base:vals}, index=timestamps)

In [ ]:
az = await get_data_frame('azimuthCalculatedAngle', 'lsst.sal.ATMCS.mount_AzEl_Encoders')

In [ ]:
measured_vel_az1 = await get_data_frame('azimuthMotor1Velocity', 'lsst.sal.ATMCS.measuredMotorVelocity')
measured_vel_az2 = await get_data_frame('azimuthMotor2Velocity', 'lsst.sal.ATMCS.measuredMotorVelocity')

In [ ]:
commanded_az = await get_data_frame('azimuth', 'lsst.sal.ATMCS.trajectory')
commanded_vel_az = await get_data_frame('azimuthVelocity', 'lsst.sal.ATMCS.trajectory')

In [ ]:
p = figure(x_axis_type='datetime', y_range=(261., 261.4), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_az.index.values, y=commanded_az['azimuth'], color='black', line_width=2, legend='Commanded Az')
p.line(x=az.index.values, y=az['azimuthCalculatedAngle'], color='lightblue', line_width=2, legend='Computed Az')
p.extra_y_ranges = {'Velocity': Range1d(start=-0.3, end=0.2)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=measured_vel_az1.index.values, y=measured_vel_az1['azimuthMotor1Velocity'], color='red', alpha=0.5, y_range_name='Velocity', legend='Measured Az Velocity Motor 1')
p.line(x=measured_vel_az2.index.values, y=measured_vel_az2['azimuthMotor2Velocity'], color='blue', alpha=0.5, y_range_name='Velocity', legend='Measured Az Velocity Motor 2')
p.line(x=commanded_vel_az.index.values, y=commanded_vel_az['azimuthVelocity'], color='black', alpha=0.5, y_range_name='Velocity', legend='Commanded Az Velocity')
p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

In [ ]:
el = await get_data_frame('elevationCalculatedAngle', 'lsst.sal.ATMCS.mount_AzEl_Encoders')

In [ ]:
measured_vel_el = await get_data_frame('elevationMotorVelocity', 'lsst.sal.ATMCS.measuredMotorVelocity')

In [ ]:
commanded_el = await get_data_frame('elevation', 'lsst.sal.ATMCS.trajectory')
commanded_vel_el = await get_data_frame('elevationVelocity', 'lsst.sal.ATMCS.trajectory')

In [ ]:
p = figure(x_axis_type='datetime', y_range=(43.4, 44.2), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Elevation (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_el.index.values, y=commanded_el['elevation'], color='black', line_width=2, legend='Commanded Elevation')
p.line(x=el.index.values, y=el['elevationCalculatedAngle'], color='lightblue', line_width=2, legend='Computed Elevation')
p.extra_y_ranges = {'Velocity': Range1d(start=-0.1, end=0.1)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=measured_vel_el.index.values, y=measured_vel_el['elevationMotorVelocity'], color='red', alpha=0.5, y_range_name='Velocity', legend='Measured El Velocity Motor')
p.line(x=commanded_vel_el.index.values, y=commanded_vel_el['elevationVelocity'], color='black', alpha=0.5, y_range_name='Velocity', legend='Commanded El Velocity')
p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)